In [1]:
from ipywidgets.widgets import HBox, VBox, Label, Textarea, Button
from IPython.display import display
from ipyfilechooser import FileChooser

RESULTS_PATH = '/home/adriana/Documents/master/hands_on_recommender_systems/Project/data/ego-twitter/results'

def select_dir(fc):
    global RESULTS_PATH
    RESULTS_PATH = fc.selected

fc = FileChooser(RESULTS_PATH)
fc.register_callback(select_dir)
fc.show_only_dirs = True
fc.title = '<b>Please chose the folder where results are located</b>'

display(fc)

FileChooser(path='/home/adriana/Documents/master/hands_on_recommender_systems/Project/data/ego-twitter/results…

In [2]:
%matplotlib inline
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

from ipywidgets.widgets import Layout, VBox, HBox, Dropdown, Label, Checkbox, Output, GridspecLayout
from IPython.display import display
import os
from ego_network import get_ego_ids
from metrics.roc import plot_roc_curve, plot_empty_roc_curve
from io_helpers.readers import read_ndarray_from_file
from models import Model, MODELS, NORMALIZATION_METHODS


selected_ego_network_id = None
selections = set()


def _update_plot_output(output):
    with output:
        output.clear_output(wait=True)
        results_dict = {}
        for (model, normalization_method) in selections:
            results_directory_for_id = os.path.join(RESULTS_PATH, str(selected_ego_network_id))
            try:
                fpr = read_ndarray_from_file(os.path.join(results_directory_for_id, f'{model.file_prefix}_{normalization_method}.fpr'))
                tpr = read_ndarray_from_file(os.path.join(results_directory_for_id, f'{model.file_prefix}_{normalization_method}.tpr'))
            except:
                continue
            results_dict[f'{model.display_name} {normalization_method}'] = {'fpr': fpr, 'tpr': tpr}
        plot_roc_curve(results_dict, selected_ego_network_id)


def _on_ids_widget_clicked(event, output: Output):
    if event['name'] != 'value':
        return
    global selected_ego_network_id
    selected_ego_network_id = event['new']
    _update_plot_output(output)


def _build_ids_widget(output: Output):
    ids = get_ego_ids(RESULTS_PATH, order_by_edges_size=False)
    global selected_ego_network_id
    selected_ego_network_id = ids[0]
    
    dropdown_widget = Dropdown(options=ids,
                               disabled=False,
                               overflow='visible',
                               layout={'width': 'max-content'})
    dropdown_widget.observe(lambda event: _on_ids_widget_clicked(event, output))
    return HBox([Label('Ego Network Id:'), dropdown_widget])



def _on_normalization_method_checkbox_click(model: Model, normalization_method: str, event, output: Output):
    if event['name'] != 'value':
        return

    value = event['new']
    if value:
        selections.add((model, normalization_method))
    else:
        selections.remove((model, normalization_method))
    _update_plot_output(output)


def _build_normalization_method_widget(model: Model, normalization_method: str, output: Output):
    widget = Checkbox(
        value=False,
        description=normalization_method,
        disabled=False,
        indent=False
    )
    widget.observe(lambda event: \
        _on_normalization_method_checkbox_click(model, 
                                                normalization_method, 
                                                event,
                                                output))
    return widget


def _build_models_widget(output: Output):
    models_layout = GridspecLayout(len(MODELS) * len(NORMALIZATION_METHODS), 2, width='350px')
    for index in range(len(MODELS)):
        model = MODELS[index]
        models_layout[index * len(NORMALIZATION_METHODS), 0] = Label(model.display_name)
        for method_index in range(len(NORMALIZATION_METHODS)):
            method = NORMALIZATION_METHODS[method_index]
            models_layout[index * len(NORMALIZATION_METHODS) + method_index, 1] =\
                _build_normalization_method_widget(model, method, output)
    return VBox([
        Label('Models:'),
        models_layout
    ])


plot_output = Output()
widgets_vbox = VBox([      
    _build_ids_widget(plot_output),
    _build_models_widget(plot_output),
], layout = Layout(
    margin = '5px'
    
))
widgets_vbox.layout.add_traits()

main_box = HBox([plot_output, widgets_vbox])
display(main_box)

with plot_output:
    plot_empty_roc_curve(selected_ego_network_id)


/home/adriana/.local/lib/python3.8/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
